<a href="https://colab.research.google.com/github/nnilayy/LLMs-And-Transformers/blob/main/Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

# Basics


## Sentiment Analysis

In [ ]:
# Sentiment Analsysis
from transformers import pipeline

classifier = pipeline("sentiment-analysis",model="distilbert-base-uncased-finetuned-sst-2-english")
classifier("horrible piece of nice trash.")

[{'label': 'POSITIVE', 'score': 0.984700083732605}]

## Zero Shot Classification

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
classifier(
    "This is a shit movie about Goverment the Transformers library",
    candidate_labels=["movies", "politics", "business"],
)

## Text Generation

In [ ]:
# Text Generation
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "Model distilgpt2 repeats the words after some times, this is because",
    max_length=100,
    num_return_sequences=2,
)
# A perfect example of model Hallucinatuion

## Mask Filling

In [ ]:
# Mask Filling Model
from transformers import pipeline

unmasker = pipeline("fill-mask", model="distilroberta-base")
unmasker("This course will teach you all about <mask> models.", top_k=2)

## Named Entity Recognition

In [ ]:
# Named Entity Recognition
from transformers import pipeline

ner = pipeline("ner",model='dbmdz/bert-large-cased-finetuned-conll03-english', grouped_entities=True)
ner("hey there, I am Nilay and the Text Generation model above was hallucinating.")

[{'entity_group': 'PER',
  'score': 0.9974254,
  'word': 'Nilay',
  'start': 16,
  'end': 21},
 {'entity_group': 'MISC',
  'score': 0.56624573,
  'word': 'Text',
  'start': 30,
  'end': 34},
 {'entity_group': 'ORG',
  'score': 0.7065201,
  'word': 'Generation',
  'start': 35,
  'end': 45}]

## QnA

In [ ]:
# Question and Answering
from transformers import pipeline

question_answerer = pipeline("question-answering",  model="distilbert-base-cased-distilled-squad")
question_answerer(
    question="Where do I sleep?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

{'score': 0.48142537474632263,
 'start': 33,
 'end': 45,
 'answer': 'Hugging Face'}

## Summarization

In [ ]:
# Summarization
from transformers import pipeline

summarizer = pipeline("summarization",model="sshleifer/distilbart-cnn-12-6")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

## Translation

In [ ]:
!pip install sentencepiece

In [ ]:
# French to English Translation
from transformers import pipeline
import sentencepiece
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

# Inside the Pipeline Function